# Anazliza bolničkih kapaciteta zemalja u svetu, tokom pandemije Covid-19



In [49]:
%matplotlib inline

import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as seb
import datetime as dt
import pandas as pd
import numpy as np

from sklearn import linear_model
from ipywidgets import interact

from IPython.core.pylabtools import figsize

In [50]:
data_dir = 'data'

country_map = {
    'US': 'United_States',
    'United States of America': 'United_States',
    'UK': 'United_Kingdom',
    'United Kingdom of Great Britain and Northern Ireland': 'United_Kingdom'
}

In [51]:
## Stopa ozbiljnosti, broj ozbilnjih slucajeva i broj kritičnih

P_SEVERE = 0.138
P_CRITICAL = 0.0188

## Statistika o broju bolničkih kreveta za zemlje

https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds

In [52]:
def getLine(line):
    return [x.strip() for x in line.split(',')]

def to_num(x):
    try:
        return float(x)
    except:
        return None

bedsFile = os.path.join(data_dir, 'hospital_beds.csv')
beds_rows = []
rows = []

with open(bedsFile, 'r') as infile:
    headers = getLine(next(infile))
    for line in infile:
        split_lines = getLine(line)
        beds_rows = dict(zip(headers, split_lines))
        rows.append(beds_rows)
        
beds = pd.DataFrame(rows)
cols = ['2013', '2014', '2015', '2016', '2017', 'Occupancy', 'ICU',]

for f in cols:
        beds[f] = beds[f].map(to_num)
        
beds['beds_p_1k'] = \
    beds['2017'].combine_first(
    beds['2016'].combine_first(
    beds['2015'].combine_first(
    beds['2014'].combine_first(
    beds['2013']
))))

beds['ICU_100k'] = beds['ICU'] #broj kreveta internzivne nege
beds = beds[['Country', 'beds_p_1k', 'ICU_100k', 'Occupancy']].copy()

beds.head()

,Country,beds_p_1k,ICU_100k,Occupancy
0,Japan,13.05,7.3,75.5
1,South_Korea,12.27,10.6,NaN
2,Russia,8.05,8.3,NaN
3,Germany,8.00,29.2,79.8
4,Austria,7.37,21.8,73.8


## Populacija za svaku zemlju

https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)

In [53]:
population = pd.read_csv(os.path.join(data_dir, 'population.csv'))
population['Country'] = population['Country or area']
population['population'] = population['Population(1 July 2019)'].map(lambda x: x.replace(',', '')).astype('float64')
population = population[['Country', 'population']].copy()
population.head()

,Country,population
0,China,1.433784e+09
1,India,1.366418e+09
2,United_States,3.290649e+08
3,Indonesia,2.706256e+08
4,Pakistan,2.165653e+08


## Spajamo podatke o broju bolničkih kreveta i populacije 

In [54]:
merged_static = beds.merge(population, on='Country', how='inner')
merged_static['beds'] = (merged_static['beds_p_1k'] / 1000.0) * merged_static['population']
merged_static['available_beds'] = merged_static['beds'] * (1.0 - merged_static['Occupancy'] / 100.0)
merged_static['icu'] = (merged_static['ICU_100k'] / 100_000.0) * merged_static['population']
merged_static['available_icu'] = merged_static['icu'] * (1.0 - merged_static['Occupancy'] / 100.0)
merged_static = merged_static[merged_static['available_icu'].notnull()]
merged_static.head()

,Country,beds_p_1k,ICU_100k,Occupancy,population,beds,available_beds,icu,available_icu
0,Japan,13.05,7.3,75.5,126860301.0,1.655527e+06,405604.097372,9260.801973,2268.896483
3,Germany,8.00,29.2,79.8,83517045.0,6.681364e+05,134963.544720,24386.977140,4926.169382
4,Austria,7.37,21.8,73.8,8955102.0,6.599910e+04,17291.764656,1952.212236,511.479606
5,Hungary,7.02,13.8,65.5,9684679.0,6.798645e+04,23455.324070,1336.485702,461.087567
6,Czech_Republic,6.63,11.6,70.1,10689209.0,7.086946e+04,21189.967245,1239.948244,370.744525


## COVID-19 podaci o broju slučajeva, broju preminulih i broj izlečenih


In [55]:
confirmed_path = os.path.join(data_dir, 'confirmed.csv')
recovered_path = os.path.join(data_dir, 'recovers.csv')
death_path = os.path.join(data_dir, 'deaths.csv')
    
    
confirmed_df = pd.read_csv(confirmed_path).drop(['Lat', 'Long'], axis=1)

#broj potvređenih slučajeva
confirmed_df =  confirmed_df.rename(columns={'Province/State': 'State', 'Country/Region': 'Country'})               
confirmed_df =  confirmed_df.melt(['State', 'Country'], var_name='Date', value_name='Confirmed')
confirmed_df =  confirmed_df.copy()

recovered_df = pd.read_csv(recovered_path).drop(['Lat', 'Long'], axis=1)

#broj izlečenih 
recovered_df = recovered_df.rename(columns={'Province/State': 'State', 'Country/Region': 'Country'})
recovered_df = recovered_df.melt(['State', 'Country'], var_name='Date', value_name='Recovered')
recovered_df =recovered_df.copy()

deaths_df = pd.read_csv(death_path).drop(['Lat', 'Long'], axis=1)

#broj preminulih
deaths_df =  deaths_df.rename(columns={'Province/State': 'State', 'Country/Region': 'Country'})
deaths_df =  deaths_df.melt(['State', 'Country'], var_name='Date', value_name='Deaths')
deaths_df =  deaths_df.copy()

MERGE_COLS = ['State', 'Country', 'Date']

covid_df =confirmed_df.merge(recovered_df, on=MERGE_COLS).merge(deaths_df, on=MERGE_COLS) 

covid_df = covid_df[covid_df[['Confirmed', 'Recovered', 'Deaths']].notnull().values.all(axis=1)]

#broj aktivnih slučajeva
covid_df['Active'] = covid_df['Confirmed'] - covid_df['Deaths'] - covid_df['Recovered'] 

#broj teških slučajeva
covid_df['Severe'] = covid_df['Active'] * P_SEVERE 

#broj kritičnoh slučajeva
covid_df['Critical'] = (covid_df['Active'] * P_CRITICAL).astype('int64') 
    
covid_df['Date'] = pd.to_datetime(covid_df['Date']).dt.date

covid_df = covid_df.groupby(['Country', 'Date']).sum().reset_index().copy()
covid_df['Date'] = pd.to_datetime(covid_df['Date'])

covid_df = covid_df[covid_df['Critical'] > 0.0].copy()


In [56]:
figsize(30, 15)
figsize(30, 15)

seb.set()
pd.set_option('display.max_columns', 50)

plt.rc('font', size=14)          
plt.rc('axes', titlesize=14)    
plt.rc('axes', labelsize=16)   
plt.rc('xtick',labelsize=14)   
plt.rc('ytick', labelsize=14)   
plt.rc('legend', fontsize=14)   
plt.rc('figure', titlesize=18) 

## Predviđanje popunjenosti bolničkih kapaciteta 

In [57]:
countries = sorted(merged_static['Country'].unique())

def show(zemlja, log, prognoza, scale):
    DAYS = 7  
    data_country = covid_df[covid_df['Country'] == zemlja]
    data_country =data_country.groupby(['Country', 'Date'])
    data_country =data_country.sum()
    data_country =data_country.reset_index()

    _, row = next(data_country.head(1).iterrows())
    first_date = row['Date']  
    
    last_week = data_country.tail(DAYS)
    _, row = next(last_week.head(1).iterrows())

    first_date_last_week = row['Date']
    lw_indices = list(range(DAYS))
    
    _, row = next(merged_static[merged_static['Country']==zemlja].iterrows())
    available_icu = row['available_icu']
         
   
    #prognoza popunjenosti kapaciteta
    if prognoza:
        
        model = linear_model.LinearRegression()
       
        for i in lw_indices:
            model.fit([[i] for i in lw_indices], np.log(last_week['Critical']))

        if available_icu and pd.notnull(available_icu):
            DAYS_PREDICT = int((np.log(available_icu) - model.intercept_)/model.coef_) + 10
        else:
            DAYS_PREDICT = 4 * DAYS

        nw_indices = list(range(DAYS, DAYS_PREDICT))
        next_week = list(np.exp(model.predict([[i] for i in nw_indices])))
       
        next_week = [scale * n for n in next_week]
            
    else:
        nw_indices = [DAYS]
        next_week = None
    
    predict_df = pd.DataFrame()
    predict_df['Date'] = pd.to_datetime([
        first_date_last_week + dt.timedelta(days=i) for i in nw_indices
    ])
    
    predict_df['Country'] = zemlja
    predict_df['Forecast'] = next_week
    
    concat_df = data_country.merge(predict_df, on=['Date', 'Country'], how='outer')
    concat_df = concat_df.assign(available_icu=available_icu)
    concat_df = concat_df.reset_index(drop=True)      
            
             
    concat_df['Date'] = concat_df['Date'].dt.date
    concat_df.set_index('Date', inplace=True)
    
    ax = concat_df['Critical'].plot(logy=log, lw=4, style='r-', use_index=True)
        
    positions = [p for p in concat_df.index if p.weekday() == 0]
        
    labels = []
    
    for l in positions:
        labels.append(l.strftime('%d.%m'))
        
     #brroj raspolozivih kreveta intenzivne nege   
    concat_df['available_icu'].plot(logy=log, lw=4, style='k--', xticks=[], use_index=True)
    
    if prognoza:
        concat_df['Forecast'].plot(logy=log, lw=4, use_index=True, style='ro')
        plt.title('Broj dana za koje će bolnice ispuniti kapacitet : {}'.format(DAYS_PREDICT - DAYS),fontsize=24,)
        
    ax.set_xticks(positions)
    ax.set_xticklabels(labels)

    ax.annotate(
        'Broj krevta intenzivne nege = {}'.format(int(available_icu)),(first_date, available_icu),fontsize=24,color='darkslategray',
         xytext=(10, -24),textcoords='offset points')
   
    plt.ylabel('Kritični slučajevi')
    plt.show()

interact(show, zemlja=countries, log=False, prognoza=False, scale=1.0)


interactive(children=(Dropdown(description='zemlja', options=('Austria', 'Belgium', 'Czech_Republic', 'Estonia…

<function __main__.show(zemlja, log, prognoza, scale)>